In [16]:
import pandas as pd

# Adding a new column with manual_score and creating a new column with manual total score (called total_score) to the cleaned data from Inspera
This file is NOT universal since I dont think that kind of task will appear again.

### Import cleaned data

In [17]:
df_cleaned = pd.read_excel("data/cleaned_data2024.xlsx", engine='openpyxl') # I will add columns to this df

### Import file with manual changed scores

In [18]:
df_manual_scores = pd.read_excel("data/2024-eksamen Analyse - FINAL - anonym.xlsx", engine='openpyxl', skiprows=1) # This is the file with manual scores
df_manual_scores = df_manual_scores.rename(columns={
            'Kandidatnr': 'candidate_id_manual',
            '1.1 tallsystem': '1.1',
            '1.2 Sortering': '1.2', 
            '1.3 ASCII hullbånd': '1.3',
            '1.4 oblig': '1.4',
            '1.5 hvilken likning': '1.5',
            '1.6 virker koden': '1.6',
            '1.7 hva blir printet': '1.7',
            '1.8 programflyt': '1.8',
            'Grand Total': 'total_score'
})

# Change the form of how the data is represented
df_manual_scores_long = df_manual_scores.melt(id_vars='candidate_id_manual',
                                  var_name='question_number',
                                  value_name='manual_score_per_question')

df_manual_scores_long.head()

,candidate_id_manual,question_number,manual_score_per_question
0,1,1.1,2.4
1,2,1.1,2.1
2,3,1.1,0.9
3,4,1.1,2.4
4,5,1.1,2.4


### Cleaned data and file with manual changed scores have different candidate ids, so I import a file that matches the ids from those two files

In [19]:
# df_manual_scores_long and df_cleaned have different candidate ids. Thtas why we need to mactch them using a third file that shows the match between the ids
df_id_match = pd.read_excel("data/2024_json_vs_anonym.xlsx", engine='openpyxl') # This is the file with ID match
df_id_match = df_id_match.rename(columns={
            'Unnamed: 0': 'candidate_id_manual',
            'Unnamed: 1': 'candidate_id'
})
df_id_match.head()

,candidate_id_manual,candidate_id
0,1,17104
1,2,17105
2,3,17106
3,4,17107
4,5,17111


### Merge

In [20]:
# Make sure question_number in df_cleaned is string
df_cleaned['question_number'] = df_cleaned['question_number'].astype(str)

# Merge first
df_merged = df_manual_scores_long.merge(df_id_match, on='candidate_id_manual', how='left')

# convert question_number to string in df_merged
df_merged['question_number'] = df_merged['question_number'].astype(str)

# Group by and aggregate
df_merged = df_merged.groupby(['candidate_id', 'question_number'], as_index=False)\
    .agg({'manual_score_per_question': 'first'})

df_merged.head()


,candidate_id,question_number,manual_score_per_question
0,17104,1.1,2.4
1,17104,1.2,3.0
2,17104,1.3,1.9
3,17104,1.4,3.0
4,17104,1.5,3.0


In [21]:
# Merge df_merged + df_cleaned on candidate_id and question_number
df_final = df_cleaned.merge(df_merged, on=['candidate_id', 'question_number'], how='left')

# help-dataframe to get unique scores
unique_scores = df_final[['candidate_id', 'question_number', 'manual_score_per_question']].drop_duplicates()

# Sum manual scores for each candidate
manual_totals = unique_scores.groupby('candidate_id')['manual_score_per_question'].sum().reset_index()
manual_totals = manual_totals.rename(columns={'manual_score_per_question': 'total_score'})

df_final = df_final.merge(manual_totals, on='candidate_id', how='left')


df_final.head()


,exam_last_modified_time,exam_start_time,extra_time_mins,incident_time_mins,candidate_id,max_question_score,question_number,question_title,question_duration (sec),auto_score_per_question,candidate_response_code,candidate_response_text,auto_total_score,manual_score_per_question,total_score
0,2024-12-11 15:57:52,2024-12-11 14:00:02,0,0,17107,2.4,1.1,Tallsystemer,85,2.4,simpleChoice_1368096164456,Heksadesimale tall,71.61,2.4,76.81
1,2024-12-11 15:57:52,2024-12-11 14:00:02,0,0,17107,2.4,1.1,Tallsystemer,85,2.4,simpleChoice_IA172399520449584402464-f255-46e6...,Oktale tall,71.61,2.4,76.81
2,2024-12-11 15:57:52,2024-12-11 14:00:02,0,0,17107,2.4,1.1,Tallsystemer,85,2.4,simpleChoice_IA17239952044953d28e157-ac4c-49db...,Titallsystemet,71.61,2.4,76.81
3,2024-12-11 15:57:52,2024-12-11 14:00:02,0,0,17107,2.4,1.1,Tallsystemer,85,2.4,simpleChoice_IA1723995204495960f601e-73d5-48f8...,Heksadesimale tall,71.61,2.4,76.81
4,2024-12-11 15:57:52,2024-12-11 14:00:02,0,0,17107,2.4,1.1,Tallsystemer,85,2.4,simpleChoice_IA17239952636882cb13f25-bd73-4352...,Titallsystemet,71.61,2.4,76.81


In [22]:
# Download the final dataframe
df_final.to_excel("data/cleaned_data_with_manual_scores2024.xlsx", index=False, engine='openpyxl')